In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('performance_results.csv')

In [3]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [5]:
df.head()

,tbt,fcp,lcp,cls,ttfb,speedindex,pagesize,jsexecutiontime,numberofrequests,numelements,pagename,url,screenshotpath,run
0,947 ms,2551 ms,13405 ms,0.414,34 ms,5973,3 KB,1998 ms,387,748,model_management,https://llmops.transwarp.io/llm/llmops/project...,llmops_transwarp_io_model_management_run1_18-0...,2
1,565 ms,2300 ms,12915 ms,0.000,112 ms,3193,3 KB,1288 ms,344,265,model_experience,https://llmops.transwarp.io/llm/llmops/project...,llmops_transwarp_io_model_experience_run1_18-0...,2
2,912 ms,1402 ms,4026 ms,0.430,10 ms,6128,3 KB,1847 ms,281,2206,model_management,https://172.17.120.207:30745/llm/dev/project/a...,172_17_120_207_30745_model_management_run1_18-...,2
3,604 ms,1397 ms,11709 ms,0.083,56 ms,4104,2 KB,1339 ms,242,1145,model_experience,https://172.17.120.207:30745/llm/dev/project/a...,172_17_120_207_30745_model_experience_run1_18-...,2
